<a href="https://colab.research.google.com/github/tomonari-masada/course2021-stats1/blob/main/plot_gmm_covariances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 混合正規分布を使ったクラスタリング（scikit-learnの実装例の解説）
* https://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_covariances.html

* scikit-learnには、混合正規分布モデルが実装されている。
* 共分散行列について制約がないモデルも使えるようになっている。
* また、異なるクラスタの共分散行列についてどの程度共通性を持たせるかも、設定できる。

* 注意：このコードでは、クラスタの重心の初期化に、ラベルの情報を使ってしまっている。
 * そのため、教師なし学習とは言えない。
 * 共分散行列についてどんな仮定すると、どんなクラスタリング結果が得られるか、を学ぶためのコード。

In [ ]:
# Author: Ron Weiss <ronweiss@gmail.com>, Gael Varoquaux
# Modified by Thierry Guillemot <thierry.guillemot.work@gmail.com>
# License: BSD 3 clause

import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np

from sklearn import datasets
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import StratifiedKFold

%config InlineBackend.figure_formats = {'png', 'retina'}

* 共分散行列について、4種類の設定を試している。


1. クラスタごとに別々の共分散行列
2. 全てのクラスタで同じ共分散行列
3. クラスタごとに別々だが対角成分のみ非ゼロの共分散行列
4. クラスタごとに別々だが単位行列の定数倍の共分散行列



In [ ]:
colors = ["navy", "turquoise", "darkorange"]

def make_ellipses(gmm, ax):
  for n, color in enumerate(colors):
    if gmm.covariance_type == "full":
      covariances = gmm.covariances_[n][:2, :2]
    elif gmm.covariance_type == "tied":
      covariances = gmm.covariances_[:2, :2]
    elif gmm.covariance_type == "diag":
      covariances = np.diag(gmm.covariances_[n][:2])
    elif gmm.covariance_type == "spherical":
      covariances = np.eye(gmm.means_.shape[1]) * gmm.covariances_[n]
    v, w = np.linalg.eigh(covariances)
    u = w[0] / np.linalg.norm(w[0])
    angle = np.arctan2(u[1], u[0])
    angle = 180 * angle / np.pi  # convert to degrees
    v = 2.0 * np.sqrt(2.0) * np.sqrt(v)
    ell = mpl.patches.Ellipse(
        gmm.means_[n, :2], v[0], v[1], 180 + angle, color=color
    )
    ell.set_clip_box(ax.bbox)
    ell.set_alpha(0.5)
    ax.add_artist(ell)
    ax.set_aspect("equal", "datalim")

* irisデータセットをクラスタリングする。
 * 25%をテストデータとして確保し、クラスタリングの結果を使って分類する。

In [ ]:
iris = datasets.load_iris()

# Break up the dataset into non-overlapping training (75%) and testing
# (25%) sets.
skf = StratifiedKFold(n_splits=4)
# Only take the first fold.
train_index, test_index = next(iter(skf.split(iris.data, iris.target)))


X_train = iris.data[train_index]
y_train = iris.target[train_index]
X_test = iris.data[test_index]
y_test = iris.target[test_index]

n_classes = len(np.unique(y_train))

* 4種類のモデルを辞書に保存しておく。

In [ ]:
# Try GMMs using different types of covariances.
estimators = {
    cov_type: GaussianMixture(
        n_components=n_classes, covariance_type=cov_type, max_iter=20, random_state=0
    )
    for cov_type in ["spherical", "diag", "tied", "full"]
}

n_estimators = len(estimators)

* クラスタリングの結果を描画する。
 * 各クラスタの正規分布の密度関数の等高線を比べてみよう。

In [ ]:
plt.figure(figsize=(3 * n_estimators // 2, 6))
plt.subplots_adjust(
    bottom=0.01, top=0.95, hspace=0.15, wspace=0.05, left=0.01, right=0.99
)


for index, (name, estimator) in enumerate(estimators.items()):
  # 重心ベクトルは、正解のラベルを使って初期化している。
  estimator.means_init = np.array(
      [X_train[y_train == i].mean(axis=0) for i in range(n_classes)]
  )
  
  # 他のパラメータは、EMアルゴリズムで推定する。
  estimator.fit(X_train)

  h = plt.subplot(2, n_estimators // 2, index + 1)
  make_ellipses(estimator, h)

  for n, color in enumerate(colors):
    data = iris.data[iris.target == n]
    plt.scatter(
        data[:, 0], data[:, 1], s=0.8, color=color, label=iris.target_names[n]
    )
  
  # Plot the test data with crosses
  for n, color in enumerate(colors):
    data = X_test[y_test == n]
    plt.scatter(data[:, 0], data[:, 1], marker="x", color=color)

  y_train_pred = estimator.predict(X_train)
  train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel()) * 100
  plt.text(0.05, 0.9, "Train accuracy: %.1f" % train_accuracy, transform=h.transAxes)

  y_test_pred = estimator.predict(X_test)
  test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel()) * 100
  plt.text(0.05, 0.8, "Test accuracy: %.1f" % test_accuracy, transform=h.transAxes)

  plt.xticks(())
  plt.yticks(())
  plt.title(name)

plt.legend(scatterpoints=1, loc="lower right", prop=dict(size=12))


plt.show()

## 問題
* 初期化に正解ラベルを使わずにクラスタリングを実行してみよう。
* その場合、クラスタリングの良し悪しを、テストデータを使ってどのように評価すればよいだろうか。